In [11]:
import requests
import pandas as pd
import json
import argparse
import pprint

In [12]:
def runGrapoQLQuery(url,query):
    headers = {"content-type":"application/json"}
    #try:
    results = requests.post(url = url, json={"query":query, "headers":headers})
    results = results.json()
    return results


In [27]:
def getSampleID(jsonthing):
    samplelist = jsonthing['samples']
    for participant in samplelist:
        sampleid = participant['participant']['participant_id']
    return sampleid

In [28]:
examplequery = """
    {
    study(phs_accession:"phs002371"){
        files(experimental_strategy_and_data_subtypes:"WXS"){
        file_name
        file_id
        md5sum
        samples{
            participant{
            participant_id
            }
        }
        }
    }
    }
    """

In [29]:
CDSGraphQLUrl="https://dataservice.datacommons.cancer.gov/v1/graphql/"
drsurl = 'drs://nci-crdc.datacommons.io/'

In [30]:
jsonres = runGrapoQLQuery(CDSGraphQLUrl, examplequery)

In [31]:
filelist = jsonres['data']['study'][0]['files']

In [24]:
headers = ['drs_uri', 'name', 'Participant ID', 'Md5sum', 'User Comment']
finaldf = pd.DataFrame(columns = headers)

In [34]:
for file in filelist:
    participantid = getSampleID(file)
    finaldf.loc[len(finaldf.index)] = [drsurl+file['file_id'], file['file_name'], participantid, file['md5sum'],'']

In [35]:
display(finaldf)

,drs_uri,name,Participant ID,Md5sum,User Comment
0,drs://nci-crdc.datacommons.io/dg.4DFC/9bfcd7b8...,6109-AS-21_S1_L005_R1_001.fastq.gz,HTA11_8920,dd83263a94c58708984c1e849742c3bd,
1,drs://nci-crdc.datacommons.io/dg.4DFC/37d757db...,0000351169.bam,HTA9_1,09108b26db110c4ffb174ce48f4836a4,
2,drs://nci-crdc.datacommons.io/dg.4DFC/7d484881...,0000267407.research.bam,HTA9_1,26c258a8220a4051c82e7d64c7013526,
3,drs://nci-crdc.datacommons.io/dg.4DFC/c8df94f3...,MAP.08622_polyp_2.rmdup.indel.recal.bam,HTA11_8622,7848d2003a8163cd5fe6bf9da80aebac,
4,drs://nci-crdc.datacommons.io/dg.4DFC/4c71a9e2...,0000269959.targeted.bam,HTA9_1,4ba73c3e73503481d62b811973c41898,
...,...,...,...,...,...
348,drs://nci-crdc.datacommons.io/dg.4DFC/aebaffb0...,5234-AS-37-TGTAGACGAA-AAGTATGCGT_S01_L005_R1_0...,HTA11_6134,18e4126ea909fba1ffb04ab80006e0c8,
349,drs://nci-crdc.datacommons.io/dg.4DFC/f8db2d7c...,5813-AS-16_S1_L005_R1_001.fastq.gz,HTA11_99999955474,4ab41969f76d00565d56002057c3dbe8,
350,drs://nci-crdc.datacommons.io/dg.4DFC/69f8d855...,5234-AS-68-CTCACCAGAA-GAACCTGACC_S01_L005_R1_0...,HTA11_866,19dd1b398783f9ce4fc7e2b8d6d3570c,
351,drs://nci-crdc.datacommons.io/dg.4DFC/db336d80...,5813-AS-12R_S1_L005_R1_001.fastq.gz,HTA11_99999959460,a49a89e5b318e66ca50ff86595929262,


In [37]:
filename = 'CDS_manifest.csv'
finaldf.to_csv(filename, sep=',', index=False)